In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
!ls '/content/drive/MyDrive/Colab Notebooks/data/image'

origin.7z.001  origin.7z.003  origin.7z.005  origin.7z.007
origin.7z.002  origin.7z.004  origin.7z.006  origin.7z.008


In [ ]:
!7z x '/content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.*'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Colab Notebooks/data/image/                                                            8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%      0% - origin/afraid_African_214.jpg  

In [5]:
!cp -r ./origin '/content/drive/MyDrive/Colab Notebooks/Extracted images/origin'

cp: cannot stat './origin': No such file or directory


In [6]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
images_path = '/content/drive/MyDrive/Colab Notebooks/Extracted images/origin'
label_path = '/content/drive/MyDrive/Colab Notebooks/data/label/label.lst'

In [8]:
df_info = pd.read_csv(label_path, sep=" ", header=None)
col_names = "image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_cofidence expression_label".split()
df_info.columns = col_names
col_names


['image_name',
 'face_id_in_image',
 'face_box_top',
 'face_box_left',
 'face_box_right',
 'face_box_bottom',
 'face_box_cofidence',
 'expression_label']

In [9]:
conf = df_info[df_info.face_box_cofidence >= 60]
conf.head()


,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
5,angry_actor_137.jpg,0,93,468,842,467,88.9519,0
9,angry_actor_150.jpg,0,56,263,376,169,81.8792,0
11,angry_actor_16.jpg,0,158,249,521,430,66.4535,0


In [10]:
conf.shape

(33375, 8)

In [12]:
image = []
labels = []
for i, row in conf.iterrows():
    img_name = row["image_name"]
    x1 = row["face_box_left"]
    x2 = row["face_box_right"]
    y1 = row["face_box_top"]
    y2 = row["face_box_bottom"]
    label = row["expression_label"]
    img_path = os.path.join(images_path, img_name)
    img = cv2.imread(img_path)
    # Check if img is not None
    if img is not None:
        # Crop the image using the provided coordinates
        cropped_img = img[y1:y2, x1:x2]
    else:
        continue
    if cropped_img is not None:
        # Resize the cropped image to a fixed size (e.g., 64x64)
        resized_face = cv2.resize(cropped_img, (64, 64))
    else:
        continue
    normalized_face = resized_face / 255.0
    image.append(normalized_face)
    labels.append(label)

In [13]:
X = np.array(image)
Y = np.array(labels)
X.shape

(17372, 64, 64, 3)

In [15]:
from sklearn.model_selection import train_test_split


In [16]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

print(len(X_train))
print(len(X_val))
print(len(X_test))


12160
2606
2606


In [17]:
from keras.utils import to_categorical

Y_train_one_hot = to_categorical(Y_train, num_classes=7)
Y_val_one_hot = to_categorical(Y_val, num_classes=7)
Y_test_one_hot = to_categorical(Y_test, num_classes=7)

In [18]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import  Conv2D, MaxPooling2D, Flatten,BatchNormalization, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [19]:
model = Sequential()

In [20]:
#First CNN LAYER
from keras.layers.core.activation import Activation
model.add(Conv2D(32,(3,3),padding = 'same', input_shape = (64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#Second CNN LAYER

model.add(Conv2D(64,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#THIRD CNN LAYER

model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))



In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1

In [23]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

datagen.fit(X_train)


In [24]:

log_dir = "/content/drive/MyDrive/Colab Notebooks/logs"
tensorboard_callback = TensorBoard(log_dir=log_dir)
checkpoint_callback = ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/saved_models/weights.{epoch:02d}.h5',
                                      save_weights_only=True,
                                      monitor='val_loss', # Choose a metric to monitor, e.g., validation loss
                                      save_best_only=False, # Set this to True if you want to save only the best model
                                      save_freq='epoch')

# Train the model using the augmented data and TensorBoard callback
history = model.fit(datagen.flow(X_train, Y_train_one_hot, batch_size=32),
                            epochs=20,
                            validation_data=(X_val, Y_val_one_hot),
                            callbacks=[tensorboard_callback])


Epoch 1/20
380/380 [==============================] - 240s 624ms/step - loss: 2.3919 - accuracy: 0.4385 - val_loss: 1.4129 - val_accuracy: 0.4754
Epoch 2/20
380/380 [==============================] - 275s 725ms/step - loss: 1.4297 - accuracy: 0.4787 - val_loss: 1.3773 - val_accuracy: 0.4747
Epoch 3/20
380/380 [==============================] - 287s 755ms/step - loss: 1.3956 - accuracy: 0.4810 - val_loss: 1.3311 - val_accuracy: 0.4754
Epoch 4/20
380/380 [==============================] - 261s 686ms/step - loss: 1.3779 - accuracy: 0.4799 - val_loss: 1.2997 - val_accuracy: 0.4992
Epoch 5/20
380/380 [==============================] - 251s 659ms/step - loss: 1.3394 - accuracy: 0.4917 - val_loss: 1.3157 - val_accuracy: 0.5027
Epoch 6/20
380/380 [==============================] - 243s 637ms/step - loss: 1.3102 - accuracy: 0.5014 - val_loss: 1.3273 - val_accuracy: 0.5115
Epoch 7/20
380/380 [==============================] - 240s 631ms/step - loss: 1.2648 - accuracy: 0.5254 - val_loss: 1.2526 -

In [25]:
Y_pred_one_hot = model.predict(X_test)
Y_pred = np.argmax(Y_pred_one_hot, axis=1)

82/82 [==============================] - 10s 121ms/step


In [29]:
Y_pred


array([3, 3, 6, ..., 3, 3, 6])

In [26]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
